## Load packages

In [31]:
import os
import glob
import shutil

import numpy as np
import geopandas as gpd
from PIL import Image

In [32]:
annotations = ("my_annotations", "all_annotations")
path_to_tile_index = "..\\data\\map_data\\train_data_tile_index.geojson"
tile_index = gpd.read_file(path_to_tile_index)

In [33]:
tile_index

,ID,layer,path,geometry
0,kolstad_190414_p4p_32_14.tif,kolstad_190414_p4p_tile_index,S:/Prosjekter/52106_SFI_SmartForest/annotated_...,"MULTIPOLYGON (((593804.525 6614447.699, 593814..."
1,kolstad_190414_p4p_32_13.tif,kolstad_190414_p4p_tile_index,S:/Prosjekter/52106_SFI_SmartForest/annotated_...,"MULTIPOLYGON (((593794.525 6614447.699, 593804..."
2,kolstad_190414_p4p_32_12.tif,kolstad_190414_p4p_tile_index,S:/Prosjekter/52106_SFI_SmartForest/annotated_...,"MULTIPOLYGON (((593784.524 6614447.699, 593794..."
3,kolstad_190414_p4p_32_11.tif,kolstad_190414_p4p_tile_index,S:/Prosjekter/52106_SFI_SmartForest/annotated_...,"MULTIPOLYGON (((593774.524 6614447.699, 593784..."
4,kolstad_190414_p4p_32_18.tif,kolstad_190414_p4p_tile_index,S:/Prosjekter/52106_SFI_SmartForest/annotated_...,"MULTIPOLYGON (((593844.527 6614447.699, 593854..."
...,...,...,...,...
3060,kolstad_190414_p4p_31_21.tif,kolstad_190414_p4p_tile_index,S:/Prosjekter/52106_SFI_SmartForest/annotated_...,"MULTIPOLYGON (((593874.528 6614457.700, 593884..."
3061,kolstad_190414_p4p_31_20.tif,kolstad_190414_p4p_tile_index,S:/Prosjekter/52106_SFI_SmartForest/annotated_...,"MULTIPOLYGON (((593864.527 6614457.700, 593874..."
3062,kolstad_190414_p4p_31_19.tif,kolstad_190414_p4p_tile_index,S:/Prosjekter/52106_SFI_SmartForest/annotated_...,"MULTIPOLYGON (((593854.527 6614457.700, 593864..."
3063,kolstad_190414_p4p_31_24.tif,kolstad_190414_p4p_tile_index,S:/Prosjekter/52106_SFI_SmartForest/annotated_...,"MULTIPOLYGON (((593904.529 6614457.700, 593914..."


In [34]:
class_names = ['tree']
class_ids = {'tree': 0}

In [35]:
for a in annotations:
    print("--------")
    print(f"Processing {a}")
    path_to_tiles = f"..\\data\\tiles\\{a}"
    path_to_annotation = f"..\\data\\annotated_data\\train\\{a}"
    annotation = gpd.read_file(f"{path_to_annotation}\\annotations.shp")
    annotation['class']='tree'
    output_dir = f"{path_to_annotation}\\annotated_tiles"
    os.makedirs(output_dir, exist_ok=True)

    tiles_imgs = glob.glob(f"{path_to_tiles}\\*.tif")
    tiles_names = [os.path.basename(path) for path in tiles_imgs]
    tile_subset = tile_index[tile_index['ID'].isin(tiles_names)]

    for i, tile in tile_subset.iterrows(): 
        # iterate through all rows in tile index
        filename = tile['ID'] # get filename

        # get tile polygon
        tile_poly = tile_index[tile_index['ID'] == filename]

        # select annotations that intersect with the ith tile
        ann_in_tile = gpd.sjoin(
            annotation,
            tile_poly,
            how = 'inner',
            predicate = 'intersects',
        ) 
        #ann_in_tile = annotations[annotations.intersects(tile.geometry)]
        
        # skip iteration if there are no annotations in a tile
        if len(ann_in_tile) == 0:
            print("No annotations.")
            continue
        
        # get image size metadata
        if not os.path.exists(f"{path_to_tiles}\\{filename}"):
            print("skip")
            continue
        print("Annotations in tile.")

        try:
            one_img= Image.open(f"{path_to_tiles}\\{filename}") # read in the first one
        except:
            print("Unable to open image.")
            continue
        one_img_array = np.array(one_img) # convert to a numpy array
        # get image width and height
        img_height_px = one_img_array.shape[0]
        img_width_px = one_img_array.shape[1]

        # Define the output file path
        output_file = f"{output_dir}\\{os.path.splitext(filename)[0]}.txt"

        # Open the output file for writing
        with open(output_file, 'w') as f:
            # Iterate through each polygon (j= 1,2,3,...) in the GeoJSON file
            for j, ann in ann_in_tile.iterrows():
                if ann['geometry'].geom_type == 'Polygon':
                    polygons = [ann['geometry']]
                else:
                    polygons = ann['geometry'].geoms

                # iterate through each vertex in the annotated bounding box
                for polygon in polygons:
                    vertices = polygon.exterior.coords[:]

                    # get tile extent
                    tile_min_x, tile_min_y, tile_max_x, tile_max_y = tile['geometry'].bounds
                    tile_width_m = round(tile_max_x - tile_min_x)
                    tile_height_m = round(tile_max_y - tile_min_y)
                    
                    # get UTM coordinates of bounding box centre
                    center_x_UTM = sum(coord[0] for coord in vertices) / len(vertices)
                    center_y_UTM = sum(coord[1] for coord in vertices) / len(vertices)

                    # convert them to coordinates relative to the tile size with origin 0, 0 in the upper left corner
                    center_x = center_x_UTM - tile_min_x
                    center_y = (tile_max_y - center_y_UTM)

                    # get tile width and height in m
                    min_x, min_y, max_x, max_y = polygon.bounds
                    width_m = max_x - min_x
                    height_m = max_y - min_y

                    # Convert the coordinates to YOLO format
                    x = center_x / tile_width_m
                    y = center_y / tile_height_m
                    w = width_m /tile_width_m    
                    h = height_m / tile_height_m

                    #######################################################################
                    # NEED TO ADD THE STUFF BELOW AND CHECK THAT IT WORKS!
                    # replace negative values with zeros
                    if x <0: x=0
                    if y <0:y=0
                    if w <0: w=0
                    if h <0: h=0

                    # replace values >1 values with 1
                    if x >1: x=1
                    if y >1:y=1
                    if w >1: w=1
                    if h >1: h=1
                    
                    # assign class ID (YOLO needs the classes to be numeric!)
                    class_id = class_ids[ann['class']]

                    # Write the annotation to the output file
                    f.write(f"{class_id} {x} {y} {w} {h}\n")
                # Copy image to output
                src_img = f"{path_to_tiles}\\{filename}"
                dest_img = f"{output_dir}\\{filename}"
                shutil.copy(src_img, dest_img)

--------
Processing my_annotations
Annotations in tile.
Annotations in tile.
Annotations in tile.
Annotations in tile.
Annotations in tile.
Annotations in tile.
Annotations in tile.
Annotations in tile.
Annotations in tile.
Annotations in tile.
Annotations in tile.
Annotations in tile.
Annotations in tile.
Annotations in tile.
Annotations in tile.
Annotations in tile.
Annotations in tile.
Annotations in tile.
Annotations in tile.
Annotations in tile.
Annotations in tile.
Annotations in tile.
Annotations in tile.
Annotations in tile.
Annotations in tile.
Annotations in tile.
Annotations in tile.
Annotations in tile.
Annotations in tile.
Annotations in tile.
Annotations in tile.
Annotations in tile.
Annotations in tile.
Annotations in tile.
Annotations in tile.
Annotations in tile.
Annotations in tile.
Annotations in tile.
Annotations in tile.
Annotations in tile.
Annotations in tile.
Annotations in tile.
Annotations in tile.
Annotations in tile.
Annotations in tile.
Annotations in tile.